In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64

# Utilities
import pandas as pd

# Your CRUD module
from animal_shelter import AnimalShelter

# ─── 1) Connect to MongoDB via your CRUD module ───────────────────────────────
username, password = "aacuser", "SNHU12345"
host, port = "nv-desktop-services.apporto.com", 33049
shelter = AnimalShelter(username, password, "AAC", "animals", host=host, port=port)

# Load data into DataFrame, drop the MongoDB _id
df = pd.DataFrame.from_records(shelter.read({}))
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

# ─── 2) Initialize Dash ───────────────────────────────────────────────────────
app = JupyterDash(__name__)

# Encode the Grazioso Salvare logo
logo_bytes = open("Grazioso Salvare Logo.png", "rb").read()
logo_b64 = base64.b64encode(logo_bytes).decode()

# ─── 3) Layout ─────────────────────────────────────────────────────────────────
app.layout = html.Div([
    # Header: logo + title + credit
    html.Div([
        html.A(html.Img(src=f"data:image/png;base64,{logo_b64}", height="80px"), href="https://www.snhu.edu", target="_blank"),
        html.H1("CS‑340 Dashboard", style={"display":"inline-block","marginLeft":"1rem"}),
        html.Div("Developed by Babatope Ayeni", style={"marginTop":"0.2rem","fontSize":"0.9rem"})
    ], style={"textAlign":"center","paddingBottom":"1rem"}),
    html.Hr(),

    # Filters
    html.Div([dcc.RadioItems(
        id="filter-type",
        options=[
            {"label":"Reset (All Dogs)",           "value":"reset"},
            {"label":"Water Rescue",               "value":"water"},
            {"label":"Mountain/Wilderness Rescue", "value":"mountain"},
            {"label":"Disaster/Tracking Rescue",   "value":"disaster"}
        ],
        value="reset",
        labelStyle={"display":"inline-block","marginRight":"2rem"}
    )], style={"padding":"1rem 0"}),
    html.Hr(),

    # DataTable
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name":c,"id":c} for c in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        sort_action="native",
        filter_action="native",
        style_table={"overflowX":"auto"},
        style_cell={"padding":"5px","textAlign":"left"}
    ),
    html.Br(), html.Hr(),

    # Pie chart & Map
    html.Div(style={"display":"flex"}, children=[
        html.Div(id="graph-id", style={"width":"50%","padding":"0 1rem"}),
        html.Div(id="map-id",   style={"width":"50%","padding":"0 1rem"})
    ])
])

# ─── 4) Query definitions ──────────────────────────────────────────────────────
FILTER_QUERIES = {
    "reset":   {},
    "water":   {"age_upon_outcome_in_weeks":{"$gte":26,"$lte":156},"breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},"sex_upon_outcome":"Intact Female"},
    "mountain":{"age_upon_outcome_in_weeks":{"$gte":26,"$lte":156},"breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},"sex_upon_outcome":"Intact Male"},
    "disaster":{"age_upon_outcome_in_weeks":{"$gte":20,"$lte":300},"breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male"}
}

# ─── 5) Callback: Table, Pie, and Map on filter selection only
@app.callback(
    Output("datatable-id", "data"),
    Output("graph-id", "children"),
    Output("map-id",   "children"),
    Input("filter-type", "value")
)
def update_all(filter_key):
    # 1) Filtered data
    sub = pd.DataFrame.from_records(shelter.read(FILTER_QUERIES[filter_key]))
    if "_id" in sub.columns:
        sub.drop(columns=["_id"], inplace=True)
    table_data = sub.to_dict("records")

    # 2) Pie chart: top 8 breeds + Other
    counts = sub['breed'].value_counts()
    top8 = counts.nlargest(8)
    others = counts.sum() - top8.sum()
    pie_df = pd.DataFrame({
        'breed': list(top8.index) + (['Other'] if others>0 else []),
        'count': list(top8.values) + ([others] if others>0 else [])
    })
    pie_fig = px.pie(pie_df, names='breed', values='count', title='Breed Distribution')
    pie_graph = dcc.Graph(figure=pie_fig)

    # 3) Map: show all filtered dogs
    if not sub.empty and {'location_lat','location_long'}.issubset(sub.columns):
        center_lat = sub['location_lat'].mean()
        center_lon = sub['location_long'].mean()
        markers = [
            dl.Marker(position=[row['location_lat'], row['location_long']], children=[
                dl.Tooltip(row.breed),
                dl.Popup(html.H4(row.name))
            ])
            for idx, row in sub.iterrows()
        ]
        map_widget = dl.Map(
            style={'width':'100%','height':'400px'},
            center=[center_lat, center_lon],
            zoom=10,
            children=[dl.TileLayer()] + markers
        )
    else:
        map_widget = html.Div("No location data.", style={'textAlign':'center','paddingTop':'2rem'})

    return table_data, pie_graph, map_widget

# ─── 6) Run server ────────────────────────────────────────────────────────────
app.run_server(mode="inline", debug=True)
